import

In [27]:
import pandas as pd

# file_path = './data/raw/Anderson%20Words.xlsx'
# df = pd.read_excel(file_path)
file_path = './data/raw/input.csv'
df = pd.read_csv(file_path)

pd.set_option('display.max_columns', None)


In [28]:
df

,Rank (Likability),Words,Likability scores
0,1,sincere,573
1,2,honest,555
2,3,understanding,549
3,4,loyal,547
4,5,truthful,545
...,...,...,...
550,551,dishonest,41
551,552,cruel,40
552,553,mean,37
553,554,phony,27


In [29]:
selected_cols = ['Words', 'Likability scores', 'Rank (Likability)']
filtered_df = df[selected_cols]
filtered_df = filtered_df.rename(columns={
    'Rank (Likability)': 'rank',
    'Words': 'personality_word',
    'Likability scores': 'total_likeability',
    
})

In [30]:
filtered_df

,personality_word,total_likeability,rank
0,sincere,573,1
1,honest,555,2
2,understanding,549,3
3,loyal,547,4
4,truthful,545,5
...,...,...,...
550,dishonest,41,551
551,cruel,40,552
552,mean,37,553
553,phony,27,554


In [31]:
filtered_df.to_csv('./data/final/actual.csv', index=False)

# For getting average predictions

In [32]:
import pandas as pd
import os

# Directory containing CSV files
directory_path = "./data/raw/predictions/"

# List all CSV files in the directory
csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

# Initialize an empty list to store dataframes
df_list = []

# Load each CSV file into a DataFrame and append it to the list
for file in csv_files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path)
    df_list.append(df)

# If there's only one file, use that directly instead of concatenating
if len(df_list) == 1:
    combined_df = df_list[0]
else:
    # Concatenate all dataframes into one if multiple files
    combined_df = pd.concat(df_list, ignore_index=True)

# Ensure that columns that should be numeric are actually numeric
# combined_df['mean_val'] = pd.to_numeric(combined_df['mean_val'], errors='coerce')
# combined_df['mean_img'] = pd.to_numeric(combined_df['mean_img'], errors='coerce')
combined_df['total_likeability'] = pd.to_numeric(combined_df['total_likeability'], errors='coerce')

# Ensure data is grouped by 'Personality_word'
# Custom aggregation for Word_valence: pick the most frequent value
def most_frequent(series):
    mode = series.mode()
    if not mode.empty:
        return mode[0]  # Return the first mode if it exists
    else:
        return None  # Return None if no mode exists

# Calculate the aggregated values
averaged_df = combined_df.groupby('personality_word', sort=False).agg({
    # 'mean_val': 'mean',              # Calculate mean for numeric columns
    # 'mean_img': 'mean',
    'total_likeability': 'mean',
    'rank': most_frequent  # Get the most frequent value (mode) for the 'rank' column
}).reset_index()

# Maintain the order of appearance of Personality_word
# Extract the order of Personality_word as they first appeared
first_appearance_order = pd.Series(combined_df['personality_word']).drop_duplicates()

# Reorder the aggregated dataframe to match the first appearance order
averaged_df = averaged_df.set_index('personality_word').loc[first_appearance_order].reset_index()

# Show the averaged dataframe
averaged_df


,personality_word,total_likeability,rank
0,sincere,540.0,1.0
1,honest,530.0,2.0
2,understanding,525.0,3.0
3,loyal,520.0,4.0
4,truthful,515.0,5.0
5,trustworthy,510.0,6.0
6,intelligent,505.0,7.0
7,dependable,500.0,8.0
8,open-minded,495.0,9.0
9,thoughtful,490.0,10.0


In [33]:
averaged_df.to_csv("./data/final/predicted.csv", index=False)